In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [37]:
import pandas as pd
from urlparse import urlparse
import tldextract
from collections import Counter
import jieba
import json
import math

In [3]:
df = pd.read_csv('/Users/nhu2000/projects/similarsearch/data/elders/Solr-elders-summary_v1-1.csv')
#df = pd.read_csv('/Users/nhu2000/Desktop/huizhou/data/huiwenhua/solr_combined/Solr-huizhou-final-title-jieba-segment_v2.csv')

In [4]:
df.info()

In [5]:
import re
match = re.compile('([a-zA-Z_0-9_=_#_&_/])') #remove letters, numbers, and special characters in the article body
#match = re.compile('([a-zA-Z_0-9])') #remove all of english letters and numbers
def classifier_filters(s):
    return match.sub(r' ', s) 

In [6]:
def filter_combined(x):
    s = x.split()
    r = [t for t in s if (t.strip()) > 0 and t.strip() not in keyword_stop_words]
    t = ' '.join(r)
    ret = classifier_filters(t)
    #print 'before -----', t
    #print 'after ------', ret
    return ret   

In [7]:
def get_columns3():
    return ('url', 'domain', 'title', 'segment', 'key_words', 'summary', 'body', 'target') 
def get_empty_columns3():
    return ('', '', '', '', '', '', '', '') 

In [8]:
idx = 0
new_df = pd.DataFrame(columns = get_columns3())

In [9]:
for i in range(0, len(df)):
    s = df.iloc[i]['title']
    if '老年' in s or '老人' in s:    
        new_df.loc[idx] = get_empty_columns3()
        new_df.loc[idx]['url'] = df.iloc[i]['url']
        new_df.loc[idx]['title'] = s
        idx += 1

In [10]:
print idx

In [11]:
jieba.load_userdict("/Users/nhu2000/projects/similarsearch/data/dict/elders_dict.txt")

In [12]:
for i in range(0, len(new_df)):
    t0 = new_df.iloc[i]['title'].decode('utf-8')
    t20 = jieba.cut(t0) #搜索引擎模式
    t2 = " ".join(t20)
    new_df.iloc[i]['segment'] = t2
    new_df.loc[i]['domain'] = df.iloc[i]['domain']
    idx += 1
    print i, idx, t2

In [13]:
with open('../data/stop_words/Baidu_Stop_Words.json') as data_file:    
    data = json.load(data_file)

In [14]:
bd_stop_words = list(data)

elders_stop_words = [u'的', u'什么', u'如何', u'？',u'有',u'好',u'要', u'、', u'价格', u'地址', u'哪些',
                    u'”', u'“', u'多', u'是', u'怎么', u'，', u'和', u'可', u'最', u'：', u'吗', u'怎样',
                    u'岁', u'这些', u'种', u'个', u'大', u'！', u'这', u'你', u'不', u'应', u':', u'与']

In [29]:
elders_stop_words = [u'的', u'与', u'在', u'大会', u'新', u'中', u'人', u'网', u'之', u'是', u'举办', u'将', u'上', u'和',
                      u'合肥', u'商报', u'晚报', u'日', u'关于', u'镇', u'月', u'及', u'开幕', u'亮相', u'工作', u'年', 
                      u'看', u'图', u'为', u'什么', u'如何', u'？',u'有',u'好',u'要', u'、', u'价格', u'地址', u'哪些',
                      u'”', u'“', u'多', u'是', u'怎么', u'，', u'和', u'可', u'最', u'：', u'吗', u'怎样',u'。', u'|',
                      u'岁', u'这些', u'种', u'个', u'大', u'！', u'这', u'你', u'不', u'应', u':', u'与', u'可以',
                    ]

In [16]:
words = new_df['segment'].str.split()

In [17]:
flatten = [item for sublist in words for item in sublist if item not in bd_stop_words and item not in elders_stop_words]

In [18]:
count = Counter(flatten)

In [19]:
top_50 = count.most_common(50)

In [20]:
ed_words, cnt = zip(*top_50)
tot = sum(cnt)
ed_fq = [float(i)*1.0/tot for i in cnt]

In [21]:
ed_dict = dict(zip(ed_words, ed_fq))

In [48]:
for t in top_50:
    print t[0], t[1]

In [30]:
global counter
counter = 0

In [24]:
#remember to load huizhou dict at first but only need to load once
def jieba_cut(x):
    global counter
    t0 = x.decode('utf-8')
    t20 = jieba.cut(t0) #搜索引擎模式
    t2 = " ".join(t20)
    print "counter---", counter
    counter += 1
    return t2

In [31]:
def map_key_words(x):
    t = jieba_cut(x)
    s = t.split()
    ret = [item for item in s if item not in bd_stop_words and item not in elders_stop_words]
    count = Counter(ret)
    top_50 = count.most_common(50)
    if len(top_50) > 0:
        words, cnt = zip(*top_50)
        tot = sum(cnt)
        ed_fq = [float(i)*1.0/tot for i in cnt]
        dictionary = dict(zip(words, ed_fq))
        #print ret
        return dictionary
    else:
        return None

In [32]:
ret = map_key_words(df.iloc[100]['body'])
for t in ret.keys():
    print t

In [33]:
df['key_words'] = df.body.map(lambda x: map_key_words(x))

In [34]:
df = df.dropna(subset =['key_words'])
df_cl = df.drop_duplicates(['url'])

In [35]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    #print intersection
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    #words = WORD.findall(text)
    #words = text.split()
    return text

In [38]:
df_cl['sim'] = df_cl.key_words.map(lambda x: get_cosine(ed_dict, x))

In [41]:
df_cl.to_csv('../data/sim/elders_sim_v1.csv', encoding='utf-8', index=False)

In [40]:
df_cl.head(100)

,url,domain,title,body,summary,key_words,sim
0,http://www.cnys.com/caipu/jiachangcai/48027.html,cnys.com,"锅包肉,锅包肉的做法,锅包肉怎么做",我们将猪肉洗净后，然后切成薄片1厘米左右。\n将切好的猪肉放入碗中，在里面加入干淀粉和少许的...,将切好的猪肉放入碗中，在里面加入干淀粉和少许的水，拌匀至猪肉裹上一层厚厚的淀粉 猪肉裹好淀粉...,"{u'搅拌': 0.0408163265306, u'锅': 0.0204081632653...",0.000000
1,http://www.cnys.com/caipu/jiachangcai/48108.html,cnys.com,"冬瓜排骨汤,冬瓜排骨汤的做法,冬瓜排骨汤怎么做","冬瓜排骨汤,冬瓜排骨汤的做法,冬瓜排骨汤怎么做\n在锅内放入少许水，放入排骨，将排骨煮一下，...",在锅内放入少许水，放入排骨，将排骨煮一下，大约10分钟左右，捞出沸水中的泡沫，在稍微煮一下 ...,"{u'砂锅': 0.0277777777778, u'锅': 0.0277777777778...",0.000000
2,http://www.cnys.com/caipu/jiachangcai/62325.html,cnys.com,"豆芽炒粉条,豆芽炒粉条的做法,豆芽炒粉条怎么做","豆芽炒粉条,豆芽炒粉条的做法,豆芽炒粉条怎么做\n将黄豆芽洗净，锅中加清水，大火烧开，将黄豆...","豆芽炒粉条,豆芽炒粉条的做法,豆芽炒粉条怎么做 将黄豆芽洗净，锅中加清水，大火烧开，将黄豆芽...","{u'开': 0.0263157894737, u'锅': 0.0263157894737,...",0.000000
3,http://www.cnys.com/caipu/jiachangcai/63018.html,cnys.com,"凉拌苦瓜,凉拌苦瓜的做法,凉拌苦瓜怎么做",将焯好的苦瓜、红椒段放入器皿中，加入盐10克，味精5克，白糖10克，香油5克。,将焯好的苦瓜、红椒段放入器皿中，加入盐10克，味精5克，白糖10克，香油5克,"{u'10': 0.117647058824, u'白糖': 0.0588235294118...",0.000000
4,http://www.cnys.com/caipu/jiachangcai/63298.html,cnys.com,"番茄菜花,番茄菜花的做法,番茄菜花怎么做",白菜花洗净，掰成小块。西红柿洗净，切成滚刀块。\n锅中加清水，大火烧开，将掰好的菜花焯熟，控...,锅中加清水，大火烧开，将掰好的菜花焯熟，控水备用 锅内下15克色拉油，大火烧至7成热，下葱姜...,"{u'开': 0.0178571428571, u'控水': 0.0178571428571...",0.000000
5,http://www.cnys.com/caipu/jiachangcai/89327.html,cnys.com,"烤蛋糕,烤蛋糕的做法,烤蛋糕怎么做",分离蛋清和蛋黄，在蛋黄里加入13g细砂糖。用打蛋器充分混合均匀。\n然后继续搅拌均匀，成功的...,搅拌蛋白至细泡时再次加入10g细砂糖，继续打发 出现纹路后加入剩余的细砂糖，继续打发 混合均...,"{u'搅拌': 0.025641025641, u'分离': 0.0128205128205...",0.000000
6,http://www.cnys.com/caipu/jiachangcai/89346.html,cnys.com,"三葱焗鲍鱼,三葱焗鲍鱼的做法,三葱焗鲍鱼怎么做","三葱焗鲍鱼,三葱焗鲍鱼的做法,三葱焗鲍鱼怎么做\n这道菜选用新鲜的小鲍鱼，处理的时候先用铁勺...",这道菜选用新鲜的小鲍鱼，处理的时候先用铁勺将整粒鲍鱼肉挖出 然后用偏硬的棕刷，将鲍鱼肉边侧的...,"{u'刀切': 0.0151515151515, u'加入': 0.015151515151...",0.000000
7,http://www.cnys.com/caipu/jiachangcai/89398.html,cnys.com,"蛋黄酥,蛋黄酥的做法,蛋黄酥怎么做",揉成面团作油皮面皮，包上保鲜膜醒面半小时左右。\n揉成面团作酥油面皮，同样包上保鲜膜醒面半小...,揉成面团作油皮面皮，包上保鲜膜醒面半小时左右 揉成面团作酥油面皮，同样包上保鲜膜醒面半小时左...,"{u'包': 0.0350877192982, u'收紧': 0.0175438596491...",0.000000
8,http://www.cnys.com/caipu/jiachangcai/89408.html,cnys.com,"青椒土豆丝,青椒土豆丝的做法,青椒土豆丝怎么做","青椒土豆丝,青椒土豆丝的做法,青椒土豆丝怎么做","青椒土豆丝,青椒土豆丝的做法,青椒土豆丝怎么做","{u'做法': 0.125, u'做': 0.125, u'青椒': 0.375, u'土豆...",0.000000
9,http://www.cnys.com/caipu/jiachangcai/89442.html,cnys.com,"手撕杏鲍菇,手撕杏鲍菇的做法,手撕杏鲍菇怎么做","手撕杏鲍菇,手撕杏鲍菇的做法,手撕杏鲍菇怎么做","手撕杏鲍菇,手撕杏鲍菇的做法,手撕杏鲍菇怎么做","{u'撕': 0.272727272727, u'手': 0.272727272727, u...",0.000000


In [51]:
idx = 0
for i in range(0, len(df_cl)):
    val = df_cl.iloc[i]['sim']
    if val > 0.15:
        print idx, val
        print df_cl.iloc[i]['title']
        print ' '.join(df_cl.iloc[i]['key_words'].keys())
        idx +=1 

In [43]:
import json
with open('../data/footprint/elders_fp.json', 'w') as fp:
    json.dump(ed_dict, fp)

In [ ]:
#update the dictionary